# Pyrope: Gemini-Driven Autonomous Vector Database

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/takurot/Pyrope/blob/main/example/pyrope_colab_demo.ipynb)

Pyrope is a self-driving vector database that integrates **Microsoft Garnet** with **Google Gemini**. 
In this notebook, we will:
1. Setup the Pyrope environment (.NET + Python).
2. Generate gRPC code for the Sidecar.
3. Start the AI Sidecar (Gemini-powered controller).
4. Start the Garnet Server.
5. Perform vector search and see Gemini's autonomous policy in action.

## 1. Setup Environment

In [ ]:
# Install .NET SDK (Required for Garnet)
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 8.0
import os
import sys
os.environ["PATH"] = f"{os.environ['HOME']}/.dotnet:" + os.environ["PATH"]

# Clone the repository (uses main branch by default)
!rm -rf Pyrope
!git clone https://github.com/takurot/Pyrope.git
%cd Pyrope
# !git checkout feature/gemini-cache-control # Uncomment if verifyng a specific branch

In [ ]:
# Install Python dependencies for AI Sidecar
!{sys.executable} -m pip install grpcio grpcio-tools google-generativeai psutil redis

## 2. Generate gRPC Code
We need to generate the Python gRPC stubs from the .proto file.

In [ ]:
# Generate Python gRPC code (Updated Path)
# Using sys.executable to ensure we use the same environment
!{sys.executable} -m grpc_tools.protoc -Isrc/Protos --python_out=src/Pyrope.AISidecar --grpc_python_out=src/Pyrope.AISidecar src/Protos/policy_service.proto

## 3. Configure Gemini API Key
Set your Gemini API key from [Google AI Studio](https://aistudio.google.com/).

In [ ]:
import getpass
import os
os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your Gemini API Key: ")
os.environ["LLM_POLICY_ENABLED"] = "true"
os.environ["GEMINI_MODEL_ID"] = "gemini-2.5-flash-lite" # Or gemini-1.5-flash

## 4. Launch Services

In [ ]:
# Compile the server first to catch build errors
!dotnet build src/Pyrope.GarnetServer --configuration Release

In [ ]:
import subprocess
import time
import socket
import sys

def wait_for_port(port, timeout=30):
    print(f"Waiting for port {port}...")
    start = time.time()
    while time.time() - start < timeout:
        try:
            with socket.create_connection(("127.0.0.1", port), timeout=1):
                return True
        except OSError:
            time.sleep(1)
    return False

print("Starting AI Sidecar (logging to sidecar.log)...")
# Check if running first
if 'sidecar_process' in locals() and sidecar_process.poll() is None:
    print("Sidecar already running.")
else:
    # Open a file for logging to avoid PIPE buffer issues and close file issues
    sidecar_log_file = open("sidecar.log", "w")
    sidecar_process = subprocess.Popen(
        [sys.executable, "-u", "src/Pyrope.AISidecar/server.py"],
        stdout=sidecar_log_file, stderr=subprocess.STDOUT, text=True
    )
    # Give Sidecar a moment
    time.sleep(2)

print("Starting Garnet Server (Pyrope)...")
if 'server_process' in locals() and server_process.poll() is None:
    print("Garnet Server already running.")
else:
    # Redirect Garnet output to file as well to prevent buffer deadlock
    garnet_log_file = open("garnet.log", "w")
    server_process = subprocess.Popen(
        ["dotnet", "run", "--project", "src/Pyrope.GarnetServer", "--configuration", "Release", "--no-build", "--", "--port", "6379", "--bind", "127.0.0.1"],
        env={**os.environ, "Auth__Enabled": "false", "PYROPE_SIDECAR_ENDPOINT": "http://127.0.0.1:50051", "Sidecar__MetricsIntervalSeconds": "2"},
        stdout=garnet_log_file, stderr=subprocess.STDOUT, text=True
    )

if wait_for_port(6379, timeout=30):
    print("✅ Services started successfully and port 6379 is open!")
else:
    print("❌ Timed out waiting for Garnet Server.")
    if server_process.poll() is not None:
        print("Server Process Exited. Check garnet.log")
    else:
        print("Server Process is still running but port is not open.")
        server_process.terminate()
    
    if sidecar_process.poll() is not None:
        print("Sidecar Process Exited.")
        sys.stdout.flush()
        # Check logs immediately
        !cat sidecar.log


## 5. Run Vector Operations
We use standard Redis client to interact with Pyrope's `VEC.*` commands.

In [ ]:
import redis
import json
import random
import struct

# Connect with increased timeout
r = redis.Redis(host='127.0.0.1', port=6379, decode_responses=False, socket_timeout=5)

TENANT = "colab_user"
INDEX = "demo_index"
DIM = 32

def float_list_to_binary(floats):
    return struct.pack(f"{len(floats)}f", *floats)

try:
    print("Pinging server...")
    print(f"Response: {r.ping()}")

    print("Adding vectors...")
    for i in range(50):
        vec = [random.random() for _ in range(DIM)]
        # VEC.ADD <tenant> <index> <id> VECTOR <blob> META <json>
        r.execute_command("VEC.ADD", TENANT, INDEX, f"v{i}", "VECTOR", float_list_to_binary(vec), "META", json.dumps({"label": i}))
    print("Vectors added.")

    print("Performing search...")
    query = [random.random() for _ in range(DIM)]
    # Correct Order: VEC.SEARCH <tenant> <index> TOPK <k> VECTOR <vec>
    results = r.execute_command("VEC.SEARCH", TENANT, INDEX, "TOPK", 5, "VECTOR", float_list_to_binary(query))
    print(f"Search Results: {results}")

    # Wait for metrics to be reported to Sidecar
    print("\n⏳ Waiting 10 seconds for metrics to be reported to Sidecar... (Metrics Interval is 2s)")
    time.sleep(10)
    print("Done waiting. You can now check the logs.")

except Exception as e:
    print(f"Error: {e}")

## 6. Check AI Sidecar Logs
Let's see how Gemini is responding to the metrics.

In [ ]:
# Flush file handles
try:
    sidecar_log_file.flush()
    garnet_log_file.flush()
except: pass

print("--- Sidecar Logs ---")
!cat sidecar.log

print("\n--- Garnet Logs (Last 20 lines) ---")
!tail -n 20 garnet.log

# Note: Processes are still running. 
# To stop them, run the Cleanup cell below.

### Understanding the Logs

In the **Sidecar Logs** above, look for the following sequence which indicates Gemini is controlling the cache:

1.  **Metric Reporting**:
    ```
    Metrics: qps=0.50 miss_rate=1.00 latency_p99_ms=50.00 ... -> Policy(ttl=300)
    ```
    The Sidecar receives metrics from Garnet (high miss rate in this example).

2.  **LLM Trigger**:
    ```
    INFO - Triggered async LLM update for key ...
    ```
    Since the metrics have changed (freshness check), the `LLMPolicyEngine` decides to ask Gemini for a new policy asynchronously.

3.  **Gemini's Decision**:
    ```
    INFO - LLM Policy Updated ... PolicyConfig(admission_threshold=0.5, ttl_seconds=3600...)
    ```
    Gemini analyzes the metrics (high miss rate) and decides to **increase the TTL** (e.g., to 3600s) to keep data in cache longer and improve the hit rate.

## 7. Cleanup (Optional)
Run this cell only when you are done with the demo.

In [ ]:
print("Stopping services...")
if 'sidecar_process' in locals() and sidecar_process.poll() is None:
    sidecar_process.terminate()
    sidecar_process.wait()

if 'server_process' in locals() and server_process.poll() is None:
    server_process.terminate()
    server_process.wait()

try:
    sidecar_log_file.close()
    garnet_log_file.close()
except:
    pass
print("Services stopped.")